In [5]:
import define_transformations_afolu as dta
import define_transformations_circular_economy as dtc
import define_transformations_energy as dte
import define_transformations_ippu as dti
import define_transformations_integrated as dtr

import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import setup_analysis as sa
import sisepuede_data_api as api
import sisepuede_models as sm
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import transformations_base_afolu as tba
import transformations_base_circular_economy as tbc
import transformations_base_energy as tbe
import transformations_base_general as tbg
import transformations_base_ippu as tbi



import sisepuede as ssp
warnings.filterwarnings("ignore")

def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_convert_primaries_to_strategy.log"))


<Logger __main__ (DEBUG)>

In [3]:
regions_lac = [
    "argentina",
    "bahamas",
    "barbados",
    "belize",
    "bolivia",
    "brazil",
    "chile",
    "colombia",
    "costa_rica",
    "dominican_republic",
    "ecuador",
    "el_salvador",
    "guatemala",
    "guyana",
    "haiti",
    "honduras",
    "jamaica",
    "mexico",
    "nicaragua",
    "panama",
    "paraguay",
    "peru",
    "suriname",
    "trinidad_and_tobago",
    "uruguay",
    "venezuela"
]



year_0_ramp = 2025
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"
fp_inputs = os.path.join(dir_calibs, "input_all_sectors.csv")
df_input = pd.read_csv(fp_inputs)
field_region = "nation"

regions_run = [
    x for x in sorted(regions_lac) if x in list(df_input[field_region].unique())
]

In [8]:

import sisepuede as ssp
sisepuede = ssp.SISEPUEDE(
    "calibrated", 
    id_str = "sisepuede_run_2023-10-06T20:10:09.932282",
    logger = logger,
    regions = regions_run,
)






# STRATS TO RUN

attr_strat = sisepuede.model_attributes.dict_attributes.get("dim_strategy_id")
dict_code_to_id = attr_strat.field_maps.get(f"strategy_code_to_{attr_strat.key}")
strat_codes_ignore = [
    # AFOLU
    "LNDU:INC_LAND_REHABILITIATION",
    "LSMM:INC_MANAGEMENT_CATTLE_PIGS",
    "LSMM:INC_MANAGEMENT_OTHER",
    "LSMM:INC_MANAGEMENT_POULTRY",
    "LSMM:INC_CAPTURE_BIOGAS",
    "TRWW:INC_COMPLIANCE_SEPTIC",
    "IPPU:DEC_OTHER_FCS",
    "IPPU:DEC_HFCS",
    "IPPU:DEC_PFCS",
    
    # ENERGY
    "TRNS:BUNDLE_DEMAND_MANAGEMENT",
    "TRNS:BUNDLE_DEMAND_MANAGEMENT_REP",
    "TRNS:BUNDLE_EFFICIENCY",
    "TRNS:BUNDLE_EFFICIENCY_REP",
    "TRNS:BUNDLE_FUEL_SWITCH",
    "TRNS:BUNDLE_FUEL_SWITCH_REP",
    "TRNS:BUNDLE_MODE_SHIFT",
    "TRNS:BUNDLE_MODE_SHIFT_REP"
]
strat_ids_drop = [
    dict_code_to_id.get(x) for x in strat_codes_ignore
]
strats_run = [x for x in attr_strat.key_values if x not in strat_ids_drop]




2023-10-07 22:01:29,087 - INFO - Successfully initialized SISEPUEDEFileStructure.
2023-10-07 22:01:29,091 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2023-10-07 22:01:29,091 - INFO - 	Setting export engine to 'sqlite'.
2023-10-07 22:01:29,114 - DEBUG - Trying to get columns for ANALYSIS_METADATA with query: select * from ANALYSIS_METADATA limit 0;
2023-10-07 22:01:29,118 - INFO - Table ANALYSIS_METADATA found in sql connection; it will not be replaced.
2023-10-07 22:01:29,119 - INFO - 	Columns successfully retrieved.
2023-10-07 22:01:29,119 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2023-10-07 22:01:29,120 - INFO - Successfully instantiated table ANALYSIS_METADATA
2023-10-07 22:01:29,122 - DEBUG - Trying to get columns for ATTRIBUTE_DESIGN with query: select * from ATTRIBUTE_DESIGN limit 0;
2023-10-07 22:01:29,125 - INFO - Table ATTRIB

# Run experiment (October 7 2023)
- response to reviewers
- Edmundo identified 5 primary ids from this set: `sisepuede_run_2023-10-06T20:10:09.932282`
- read these in, generate inputs, and map to templates


In [ ]:
# read in data frame
fp_primaries_to_strategies = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/experiments_and_analysis/sisepuede_run_2023-10-06T20;10;09.932282/primary_ids_from_experiment_as_strategies;selection_for_uncertainty.csv"
df_primaries_to_strategies = pd.read_csv(fp_primaries_to_strategies)


# next, map primaries to new stratgy ids
primaries = sorted(list(df_primaries_to_strategies[sisepuede.key_primary]))
dict_primaries_to_strategies = dict((x, 5011 + i) for i, x in enumerate(primaries))




def get_lhs_trajes(
    dict_filt: dict,
    region: str,
    sisepuede: ssp.SISEPUEDE,
) -> Tuple[pd.Series, pd.Series, bool]:
    """
    Get LHS trajectories for input to generator
    """
    future_trajectories_cur = sisepuede.experimental_manager.dict_future_trajectories.get(region)
    lhs_design_cur = sisepuede.experimental_manager.dict_lhs_design.get(region)
    region_out = sisepuede.get_output_region(region)

    design = dict_filt.get(sisepuede.key_design)
    future = dict_filt.get(sisepuede.key_future)
    strategy = dict_filt.get(sisepuede.key_strategy)

    df_lhs_l, df_lhs_x = lhs_design_cur.retrieve_lhs_tables_by_design(design, return_type = pd.DataFrame)

    # reduce lhs tables - LEs
    df_lhs_l = (
        df_lhs_l[df_lhs_l[sisepuede.key_future].isin([future])] 
        if (df_lhs_l is not None) 
        else df_lhs_l
    )
    # Xs
    df_lhs_x = (
        df_lhs_x[df_lhs_x[sisepuede.key_future].isin([future])] 
        if (df_lhs_x is not None) 
        else df_lhs_x
    )


    ##  GENERATE INPUT BY FUTURE

    # determine if baseline future and fetch lhs rows
    base_future_q = (future == sisepuede.baseline_future)
    lhs_l = (
        df_lhs_l[df_lhs_l[sisepuede.key_future] == future].iloc[0] 
        if ((df_lhs_l is not None) and not base_future_q) 
        else None
    )
    lhs_x = (
        df_lhs_x[df_lhs_x[sisepuede.key_future] == future].iloc[0] 
        if ((df_lhs_x is not None) and not base_future_q) 
        else None
    )
    
    tup_out = lhs_l, lhs_x, base_future_q
    
    return tup_out



df_out = []

for region in regions_run[0:2]:
    
    lhs_design = sisepuede.experimental_manager.dict_lhs_design.get(region)
    ftc = sisepuede.experimental_manager.dict_future_trajectories.get(region)
    
    for primary_id in primaries:
        
        strat = dict_primaries_to_strategies.get(primary_id)
        dict_filt_cur = sisepuede.odpt_primary.get_dims_from_key(
            primary_id,
            return_type = "dict",
        )
        
        
        lhs_l, lhs_x, base_future_q = get_lhs_trajes(
            dict_filt_cur,
            region,
            sisepuede,
        )

        # build inputs
        df_in = ftc.generate_future_from_lhs_vector(
            lhs_x, 
            df_row_lhc_sample_l = lhs_l,
            future_id = None, 
            baseline_future_q = base_future_q,
        )
        
        df_in[sisepuede.key_strategy] = strat
        df_in[sisepuede.key_region] = region
        
        df_out.append(df_in)
        
        print(f"primary_id {primary_id} in region {region} done.")
    
        



In [ ]:
df_out